In [ ]:
import pandas as pd

import numpy as np

import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import string

In [ ]:
vocab_size = 5000
max_len = 15

In [ ]:
df = pd.read_csv("flickr8k/captions")

In [ ]:
df.head(5)

In [ ]:
df["caption"] = df.caption.apply(lambda line:  word_tokenize(line.lower()) )

In [ ]:
df.head(5)

In [ ]:
words = []

for line in df.caption:
    words.extend(line)

In [ ]:
len(words)

In [ ]:
words, counts = np.unique(words, return_counts=True)

In [ ]:
len(words)

In [ ]:
words = words[counts.argsort()[-vocab_size:]]

In [ ]:
bad = stopwords.words("english") + list(string.punctuation)

In [ ]:
words = [word for word in words if word not in bad]

In [ ]:
vocab_size = len(words)

In [ ]:
vocab_size

In [ ]:
vocab = dict(zip(words, range(1, vocab_size + 1)))

In [ ]:
rev_vocab = dict(zip(range(1, vocab_size + 1), words))

In [ ]:
sent = df.caption[2000]

In [ ]:
" ".join(sent[1:-1])

In [ ]:
" ".join([word for word in sent[1:-1] if word in vocab])

In [ ]:
# line = random.choice(df.caption)
# line = [vocab[word] for word in line if word in vocab]

# line

In [ ]:
def data_gen(lines, batch_size=32):
    
    n = 0
    
    X, Y = [], []
    
    while True:
        
        line = random.choice(lines)
        line = [vocab[word] for word in line if word in vocab]
        if len(line) > 4:
            index = random.randint(0, len(line)-1)
            X.append(line[:index])
            Y.append(to_categorical(line[index], num_classes=vocab_size+1))
            n += 1

            if n % batch_size == 0:
                X_mod = pad_sequences(X, maxlen=max_len)
                y_mod = np.array(Y)
                X, Y = [], []
                yield [X_mod, y_mod]

    

In [ ]:
gen = data_gen(df.caption)

In [ ]:
data = next(gen)

In [ ]:
data[1].shape

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

In [ ]:
in_layer = Input(shape=(max_len,))
embedding = Embedding(input_dim=vocab_size+1, output_dim=10)(in_layer)
rnn = LSTM(units=2000, activation="relu")(embedding)
d1 = Dense(units=1000, activation="relu")(rnn)
d2 = Dense(units=500, activation="relu")(d1)
out = Dense(units=vocab_size+1, activation="softmax")(d2)

In [ ]:
model = Model(inputs=in_layer, outputs=out)

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
gen = data_gen(df.caption, batch_size=2000)

for i in range(5):
    data = next(gen)
    model.fit(data[0], data[1])


In [ ]:
line = df.caption[200][:2]


In [ ]:
line

In [ ]:
line = [vocab[word] for word in line if word in vocab]

In [ ]:
X_test = pad_sequences([line], maxlen=max_len)

In [ ]:
model.predict(X_test).argmax(axis=1)

In [ ]:
rev_vocab[4884]